# AirLLM使用4G显存即可在70B大模型上进行推理
- 70B模型需要130G显存

## 1、分层推理（Layer-wise inference）
-  在推理过程中，层按顺序执行，上一层的输出是下一层的输入，一次只执行一个层。因此，完全没有必要将所有层都保存在GPU内存中。我们可以在执行该层时从磁盘加载所需的任何层，进行所有计算，然后完全释放内存。这样，每层所需的GPU内存仅为一个transformer层的参数大小，即整个模型的1/80，约1.6GB。
- 此外，一些输出缓存也存储在GPU内存中，最大的是KV缓存，以避免重复计算。对于70B模型，这个KV缓存大小大约是：
    - 2*input_length*num_layers*num_heads*vector_dim*4
    - 输入长度为100时，此缓存=2\*100\*80\*8\*128\*4=30MB GPU内存。

## 2、Flash attention
- 将计算拆分为多个小块，逐块计算，并将内存减少到一个块的大小

## 3、模型文件共享
- 对原始的HuggingFace模型文件进行预处理，并对其进行分层分割。
- 对于存储，使用安全张量技术(https://github.com/huggingface/safetensors)。Safetensor确保存储格式和内存中格式紧密匹配，并使用内存映射进行加载以最大限度地提高速度。

## 4、元设备（Meta Device）
- 使用HuggingFace Accelerate提供的Meta Device功能https://huggingface.co/docs/accelerate/usage\\_guides/bigh\\_modeling来实施。Meta Device是一种专门为运行超大型模型而设计的虚拟设备。当您通过Meta Device加载模型时，模型数据实际上并没有被读入，只是加载了代码，内存使用率为0。
- 在执行过程中，您可以将模型的部分内容从Meta Device动态转移到CPU或GPU等真实设备。只有到那时，它才真正加载到内存中。
- 使用init_empty_weights（）可以通过Meta Device加载模型

In [ ]:
from accelerate import init_empty_weights
with init_empty_weights():
    my_model = ModelClass(...)

## 5、开源项目
上述所有技术已经集成到AirLLM  
https://github.com/lyogavin/airllm

In [ ]:
!pip install airllm

In [ ]:
from airllm import AutoModel

MAX_LENGTH = 128
# could use hugging face model repo id:
model = AutoModel.from_pretrained("garage-bAInd/Platypus2-70B-instruct")
# model = AutoModel.from_pretrained("unsloth/Meta-Llama-3.1-405B-Instruct-bnb-4bit")


# or use model's local path...
# model = AutoModel.from_pretrained("/home/ubuntu/.cache/huggingface/hub/models--garage-bAInd--Platypus2-70B-instruct/snapshots/b585e74bcaae02e52665d9ac6d23f4d0dbc81a0f")

input_text = [
    'What is the capital of United States?',
    # 'I like',
]

input_tokens = model.tokenizer(input_text,
                               return_tensors="pt",
                               return_attention_mask=False,
                               truncation=True,
                               max_length=MAX_LENGTH,
                               padding=False)

generation_output = model.generate(
    input_tokens['input_ids'].cuda(),
    max_new_tokens=20,
    use_cache=True,
    return_dict_in_generate=True)

output = model.tokenizer.decode(generation_output.sequences[0])

print(output)